In [ ]:
import os
from lxml import etree
import pandas as pd

In [ ]:
# Pega todos os arquivos XML de uma pasta e os prepara para serem lidos
def carregar_arquivos(caminho_da_pasta):
    arquivos_xml_carregados = [] # Lista para guardar os XMLs lidos

    # Vê se a pasta existe. Se não, avisa e não faz mais nada
    if not os.path.isdir(caminho_da_pasta):
        print(f"Erro: A pasta '{caminho_da_pasta}' não foi encontrada.")
        return arquivos_xml_carregados

    # Verifica cada arquivo na pasta.
    for nome_arquivo in os.listdir(caminho_da_pasta):
        # Pega os arquivos que terminam com ".xml"
        if nome_arquivo.endswith(".xml"):
            # Monta o caminho completo para o arquivo
            caminho_completo_arquivo = os.path.join(caminho_da_pasta, nome_arquivo)

            # Tenta abrir e ler o XML
            try:
                tree = etree.parse(caminho_completo_arquivo) # Lê o XML
                arquivos_xml_carregados.append(tree) # Adiciona à lista
                print(f"Arquivo '{nome_arquivo}' carregado") # Avisa que carregou

            # Se o XML estiver "quebrado" ou com erro, avisa
            except etree.XMLSyntaxError as e:
                print(f"Erro ao ler '{nome_arquivo}': O arquivo XML parece estar com problemas. Detalhes: {e}")
            # Se der outro problema ao ler o arquivo, também avisa
            except Exception as e:
                print(f"Ocorreu um problema inesperado ao processar '{nome_arquivo}': {e}")

    return arquivos_xml_carregados # Devolve a lista de XMLs lidos

In [ ]:
dados_nf = [] # Lista vazia onde vamos guardar as informações que queremos (Nome e CNPJ)
caminho = "/content/drive/MyDrive/Colab Notebooks/LeitorXML/pasta_xml" # Onde os arquivos XML de notas fiscais estão guardados

xml_trees = carregar_arquivos(caminho) # Usa a função para carregar todos os arquivos XML da pasta

ns = {'nfe': 'http://www.portalfiscal.inf.br/nfe'}

for tree in xml_trees:
    # Procura a parte do XML que fala sobre quem emitiu a nota (o 'emitente')
    emitente = tree.xpath('/nfe:NFe/nfe:infNFe/nfe:emit', namespaces=ns)

    # Se a parte do emitente foi encontrada no XML:
    if emitente:
        nota_emit = emitente[0] # Pega o primeiro (e geralmente único) emitente encontrado

        # Agora, dentro dessa parte do emitente, busca o CNPJ e o Nome
        cnpj_element = nota_emit.find('nfe:CNPJ', namespaces=ns)
        nome_element = nota_emit.find('nfe:xNome', namespaces=ns)

        # Pega o texto do CNPJ e do Nome
        # Verifica se encontrou o elemento antes de pegar o texto, pra não dar erro
        cnpj = cnpj_element.text if cnpj_element is not None else None
        nome = nome_element.text if nome_element is not None else None

        # Guarda o Nome e o CNPJ na lista de dados
        dados_nf.append({'Nome': nome, 'CNPJ': cnpj})
    else:
        # Se não achou a parte do emitente em um XML, avisa
        print(f"Atenção: A informação do 'emitente' não foi encontrada em uma das Notas Fiscais. Verifique o arquivo XML.")

# Transforma a lista de dados (Nome e CNPJ) em uma tabela organizada (DataFrame)
df = pd.DataFrame(dados_nf)
df.to_excel(f'{caminho}/informacoes_xml.xlsx', index = False) # Salva na pasta que contém os XML, uma planilha do excel com as informações obtidas